In [ ]:


import pandas as pd
import numpy as np
import os

import openai
import json

import psycopg2
import warnings
from openai import OpenAI
import supabase
from supabase import create_client, Client
from sklearn.metrics.pairwise import cosine_similarity
#from openai.embeddings_utils import cosine_similarity
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import pandas as pd
from openai import OpenAI
client = OpenAI(api_key='')

- make qa

- q+a 합치기

In [ ]:
df = pd.read_parquet('./qanew2.parquet') #parquet path
qalist=[]

for row in range(len(df)):
    t=str(df['query'][row])+str(df['generation_gt'][row])
    qalist.append(t)

    df2=pd.DataFrame(qalist, columns=["qa"])

df2.to_csv('./qadata.csv')

In [ ]:
for i in df2['qa']:
    print(i)

- 영어

In [ ]:
from googletrans import Translator
translator = Translator()


In [ ]:
for i in range(len(df2)):
    df2['qa'][i]=translator.translate(df2['qa'][i], src='en', dest='ko').text

### .

- pdf를 읽어와서 각 문장을 df로 저장

In [ ]:
import fitz  # PyMuPDF
import pandas as pd

def extract_text_from_pdf(pdf_file):

    # Open the provided PDF file
    document = fitz.open(pdf_file)

    text_data = []

    # Iterate through each page in the PDF
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text()

        # Split text into sentences
        sentences = text.split('\n\n')  # Assuming sentences are separated by newlines

        for sentence in sentences:
            if sentence.strip():  # Check if the sentence is not empty
                text_data.append({'qa': sentence.strip()})

    # Create a DataFrame from the collected text data
    df = pd.DataFrame(text_data)

    return df

# Example usage:
pdf_file = '한국화재보험협회 웹진 2.pdf'  # Replace with your PDF file path
df = extract_text_from_pdf(pdf_file)

# Print the DataFrame
print(df)
df2=df.copy()

- embedding

In [ ]:


def get_embedding(text, model="text-embedding-3-small"):
   text = text = str(text).replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df2['embedding'] = df2['qa'].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df2.to_csv('./embed.csv', index=False)

- supabase에 업로드

In [ ]:
# Supabase URL과 API 키 설정
supabase_url = ""
supabase_key = ""
# Supabase 클라이언트 생성
supabase: Client = create_client(supabase_url, supabase_key)

In [ ]:
db_params = {
    'dbname': 'postgres',
    'user': '',
    'password': '',
    'host': '',
    'port': ''
}

# Create table using psycopg2
conn = psycopg2.connect(**db_params)
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS embeddings_0714 (
        id SERIAL PRIMARY KEY,
        text TEXT NOT NULL,
        embedding FLOAT8[] NOT NULL
    );
""")
conn.commit()
cur.close()
conn.close()

# Insert embeddings into the Supabase table
for _, row in df2.iterrows():
    response = supabase.table("embeddings_0714").insert({
        "text": str(row['qa']),
        "embedding": row['embedding']  # Directly use the list
    }).execute()
    print(response)

- 영어

In [ ]:
import PyPDF2
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pandas as pd
import re

In [ ]:
translator.translate(p, src='en', dest='ko').text for p in paragraphs